In [1]:
from completed_missing_data import completed_missing_data, convert_historical_date_to_date_difference, remove_described_col, map_target
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score

In [2]:


train_org = pd.read_csv('data/train.csv')
train = completed_missing_data(train_org)
train = convert_historical_date_to_date_difference(train)
train = map_target(train)

#train_with_league_id = pd.concat([train, pd.get_dummies(train['league_id'],prefix='league_')], axis=1)

# train = remove_described_col(train)
# train_with_league_id = remove_described_col(train_with_league_id)



/home/andrzej/PycharmProjects/Physionet_Challenge_2021/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


correction col type..
percent of object with nan value:  42.20
remove coach columns..
percent of object with nan value:  18.64
remove object with more that  col 80 empty...
percent of object with nan value:  18.64
remove 9 and 10 day historii
percent of object with nan value:  15.11
remove all nan data
percent of object with nan value:  15.11


In [4]:
test_org = pd.read_csv('data/test.csv')
test = completed_missing_data(test_org)
test = convert_historical_date_to_date_difference(test)
test = remove_described_col(test)
test

correction col type..
percent of object with nan value:  39.35
remove coach columns..
percent of object with nan value:  13.93
remove object with more that  col 80 empty...
percent of object with nan value:  13.93
remove 9 and 10 day historii
percent of object with nan value:  11.15
remove all nan data
percent of object with nan value:  11.15


,league_id,is_cup,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_match_date_5,home_team_history_match_date_6,home_team_history_match_date_7,home_team_history_match_date_8,...,away_team_history_opponent_rating_7,away_team_history_opponent_rating_8,away_team_history_league_id_1,away_team_history_league_id_2,away_team_history_league_id_3,away_team_history_league_id_4,away_team_history_league_id_5,away_team_history_league_id_6,away_team_history_league_id_7,away_team_history_league_id_8
0,755,0,71,97,70,96,143,98,120,288,...,4.922229,6.796484,755.0,755.0,755.0,755.0,755.0,755.0,755.0,755.0
1,743,0,166,169,168,143,360,94,193,72,...,4.141575,5.056690,743.0,743.0,743.0,743.0,743.0,743.0,743.0,743.0
3,1314,0,48,98,46,48,918,72,97,95,...,3.746925,4.952475,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0
4,1314,0,48,98,48,48,917,68,100,95,...,4.453211,8.787767,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0
5,1314,0,48,98,46,50,893,96,96,119,...,8.730013,6.617900,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72706,776,0,145,163,169,190,122,216,142,219,...,4.494093,3.665037,776.0,776.0,776.0,776.0,776.0,776.0,776.0,776.0
72707,636,0,141,98,286,120,96,144,95,97,...,6.695206,6.109570,636.0,636.0,636.0,636.0,636.0,636.0,636.0,636.0
72708,1363,0,913,765,1875,93,450,1452,77,1097,...,2.612821,5.000000,1363.0,1363.0,1126.0,1126.0,1126.0,1126.0,1126.0,1126.0
72709,648,0,75,92,71,72,77,69,71,70,...,4.382062,5.823854,648.0,648.0,648.0,648.0,648.0,648.0,648.0,648.0


# Dodanie cech
- srednia ilosc goli  na mecz w lidzie 
- sredni ranking drużyn w lidzie
- sredni ilosc meczow wygranych w lidze
- intensywność



In [3]:
home_mean_ratting_all =pd.DataFrame()
for i in range(1,9):
    for home_or_away in ['home', 'away']:
        last_match_columns  =  [x for x in train.columns  if  str(i) in x]
        rating_col_name = home_or_away + '_team_history_league_id_'+str(i)
        home_mean_ratting = train[last_match_columns].groupby(by=rating_col_name).mean()
        # train['league_name']
        home_mean_ratting['mean_ratting_'+home_or_away +'_' + str(i)] = (home_mean_ratting[home_or_away +'_team_history_rating_'+str(i)] + home_mean_ratting[home_or_away+'_team_history_opponent_rating_'+str(i)]) / 2
        home_mean_ratting_all = home_mean_ratting_all.append(home_mean_ratting[ 'mean_ratting_'+home_or_away+ '_'+str(i)])

        team_history_goal = home_or_away + '_team_history_goal_'+str(i)
        team_history_opponent_goal = home_or_away + '_team_history_opponent_goal_'+str(i)

        train[home_or_away + '_team_history_target_' + str(i)] = np.sign(train[team_history_goal] - train[team_history_opponent_goal])





home_mean_ratting_all =home_mean_ratting_all.mean().reset_index().rename(columns = {'index': 'league_id',0: 'league_id_ratting'})

train = train.merge(home_mean_ratting_all,on ='league_id').sort_values(by='league_id')
home_mean_ratting_all

,league_id,league_id_ratting
0,2.0,6.805677
1,5.0,6.524094
2,8.0,6.374938
3,9.0,5.717973
4,12.0,5.817057
...,...,...
830,782.0,9.903354
831,977.0,4.259858
832,1194.0,5.092400
833,1629.0,5.653429


In [4]:
train = remove_described_col(train)
# %%
train

,target,is_cup,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_match_date_5,home_team_history_match_date_6,home_team_history_match_date_7,home_team_history_match_date_8,...,away_team_history_target_4,home_team_history_target_5,away_team_history_target_5,home_team_history_target_6,away_team_history_target_6,home_team_history_target_7,away_team_history_target_7,home_team_history_target_8,away_team_history_target_8,league_id_ratting
61326,0,1,99,71,406,98,99,69,98,115,...,1.0,1.0,1.0,1.0,1.0,-1.0,0.0,-1.0,1.0,6.805677
61329,0,1,144,75,96,93,314,93,98,72,...,1.0,1.0,-1.0,1.0,1.0,0.0,1.0,1.0,-1.0,6.805677
61328,0,1,144,104,66,119,290,93,169,120,...,0.0,1.0,-1.0,1.0,1.0,1.0,1.0,0.0,1.0,6.805677
61327,1,1,80,64,98,95,97,406,100,789,...,1.0,0.0,-1.0,1.0,0.0,1.0,1.0,-1.0,-1.0,6.805677
61325,-1,1,98,94,95,196,91,192,169,149,...,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,0.0,6.805677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93939,0,1,547,1344,463,165,211,168,464,208,...,-1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,7.900250
93938,-1,1,545,1133,715,168,168,168,505,168,...,-1.0,0.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,1.0,7.900250
93937,1,1,504,1344,504,168,168,168,505,168,...,-1.0,0.0,-1.0,-1.0,0.0,-1.0,1.0,0.0,1.0,7.900250
93936,1,1,504,1135,712,168,168,168,505,168,...,1.0,-1.0,-1.0,0.0,-1.0,0.0,0.0,0.0,-1.0,7.900250


In [5]:
for columns in train.columns:
    print(columns, train[columns].dtype)

target int64
is_cup int64
home_team_history_match_date_1 int64
home_team_history_match_date_2 int64
home_team_history_match_date_3 int64
home_team_history_match_date_4 int64
home_team_history_match_date_5 int64
home_team_history_match_date_6 int64
home_team_history_match_date_7 int64
home_team_history_match_date_8 int64
home_team_history_is_play_home_1 float64
home_team_history_is_play_home_2 float64
home_team_history_is_play_home_3 float64
home_team_history_is_play_home_4 float64
home_team_history_is_play_home_5 float64
home_team_history_is_play_home_6 float64
home_team_history_is_play_home_7 float64
home_team_history_is_play_home_8 float64
home_team_history_is_cup_1 float64
home_team_history_is_cup_2 float64
home_team_history_is_cup_3 float64
home_team_history_is_cup_4 float64
home_team_history_is_cup_5 float64
home_team_history_is_cup_6 float64
home_team_history_is_cup_7 float64
home_team_history_is_cup_8 float64
home_team_history_goal_1 float64
home_team_history_goal_2 float64
home

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

for data, name in  zip([train],['train'] ):
    X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0], test_size=0.33,
                                                        random_state=42)
    pipe = make_pipeline(StandardScaler(), LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr'))

    pipe.fit(X_train, y_train)
    results = pipe.predict(X_train)
    print(f'accuracy: {pipe.score(X_test, y_test):.02f}')
    print(name, results)
# drzewach, svm
# kolumny
# kaggle
#

accuracy: 0.49
train [-1  1 -1 ...  1  1 -1]


# Dobór cech

In [28]:
X_train, X_test, y_train, y_test = train_test_split(train.iloc[:, 1:], train.iloc[:, 0], test_size=0.33,
                                                    random_state=42)
model = make_pipeline(StandardScaler(), LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr'))

grouped_columns = []
last_value = None
for column in X_train.columns:
    if grouped_columns and column[:-3] in last_value:
        grouped_columns[-1].append(column)
    else:
        grouped_columns.append([column])
    last_value = column



def  features_selection(grouped_columns):
    the_best_features = []
    the_best_score = 0
    for L in range(0, 1 + len(grouped_columns)):
        errors = []
        for value in grouped_columns:

            if value in the_best_features:
                continue
            elif type(value) == list:
                if value[0] in the_best_features:
                    continue

            tmp_the_best_feuteures = the_best_features.copy()

            if type(value) == list:
                tmp_the_best_feuteures.extend(value)  # dla weekday, mnths, seazon
            else:
                tmp_the_best_feuteures.append(value)

            error = np.mean(cross_val_score(model, X_train[tmp_the_best_feuteures], y_train))
            errors.append((tmp_the_best_feuteures, error))

        errors = [(k, v) for k, v in sorted(errors, key=lambda item: item[1])]

        if errors[-1][
            1] < the_best_score and the_best_score != 0:  # Jesli  dodanie cechy zwieksza bład, to zakoncz algorytm
            break

        the_best_features = errors[-1][0]
        the_best_score = errors[-1][1]

    print('the_best_feuteures\n', the_best_features)
    print('the_best_cross_value score\n', the_best_score)
    return  the_best_features, the_best_score
the_best_features, the_best_score = features_selection(grouped_columns)
the_best_features

the_best_feuteures
 ['away_team_history_rating_1', 'away_team_history_rating_2', 'away_team_history_rating_3', 'away_team_history_rating_4', 'away_team_history_rating_5', 'away_team_history_rating_6', 'away_team_history_rating_7', 'away_team_history_rating_8', 'home_team_history_rating_1', 'home_team_history_rating_2', 'home_team_history_rating_3', 'home_team_history_rating_4', 'home_team_history_rating_5', 'home_team_history_rating_6', 'home_team_history_rating_7', 'home_team_history_rating_8', 'home_team_history_opponent_goal_1', 'home_team_history_opponent_goal_2', 'home_team_history_opponent_goal_3', 'home_team_history_opponent_goal_4', 'home_team_history_opponent_goal_5', 'home_team_history_opponent_goal_6', 'home_team_history_opponent_goal_7', 'home_team_history_opponent_goal_8', 'away_team_history_opponent_goal_1', 'away_team_history_opponent_goal_2', 'away_team_history_opponent_goal_3', 'away_team_history_opponent_goal_4', 'away_team_history_opponent_goal_5', 'away_team_history

In [30]:
'is_cup' in the_best_features

False

In [ ]:
grouped_columns = []
for column in sorted(X_train.columns.values, key=lambda  x: x[-1]):
    if grouped_columns and column[:-3] in last_value:
        grouped_columns[-1].append(column)
    else:
        grouped_columns.append([column])
    last_value = column

model = make_pipeline(StandardScaler(), LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr'))

the_best_features, the_best_score = features_selection(grouped_columns)
the_best_features